# Sprawozdanie 4 - Własne Środowisko
Autorzy: Kacper Cienkosz, Miłosz Dubiel

## Gra "Zgadnij liczbę"

### Opis:

Gra dwóch graczy: gracz i komputer.

Gracz wybiera przedział liczb, np. od 1 do 100.

Komputer losuje liczbę z tego przedziału, której gracz musi się domyślić.

Gracz próbuje zgadnąć liczbę, a komputer informuje, czy podana liczba jest za duża, za mała, czy trafiona.

### Implementacja:

1. Inicjalizacja gry:
    * Gracz wybiera przedział liczb.
    * Komputer losuje liczbę z tego przedziału.

2. Rozpoczęcie rundy:
    * Gracz podaje swoją propozycję liczby.
    * Komputer sprawdza, czy podana liczba jest prawidłowa:
        - Jeśli liczba jest trafiona, komputer informuje o tym gracza, a gra kończy się.
        - Jeśli liczba jest za duża lub za mała, komputer daje odpowiednią wskazówkę.

3. Powtórzenie rundy, aż gracz zgadnie liczbę.

4. Podanie wyniku, czy gracz zgadł liczbę.

Ta gra wymaga od gracza logicznego myślenia i podejmowania decyzji na podstawie informacji zwrotnych, co czyni ją ciekawą do eksperymentowania z różnymi strategiami uczenia ze wzmocnieniem.

In [ ]:
import numpy as np

import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import register
from tqdm import tqdm
from random import uniform

In [7]:
# fmt: off

# %%
# Declaration and Initialization
# ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#
# Our custom environment will inherit from the abstract class
# ``gymnasium.Env``. You shouldn’t forget to add the ``metadata``
# attribute to your class. There, you should specify the render-modes that
# are supported by your environment (e.g. ``"human"``, ``"rgb_array"``,
# ``"ansi"``) and the framerate at which your environment should be
# rendered. Every environment should support ``None`` as render-mode; you
# don’t need to add it in the metadata. In ``GridWorldEnv``, we will
# support the modes “rgb_array” and “human” and render at 4 FPS.
#
# The ``__init__`` method of our environment will accept the integer
# ``size``, that determines the size of the square grid. We will set up
# some variables for rendering and define ``self.observation_space`` and
# ``self.action_space``. In our case, observations should provide
# information about the location of the agent and target on the
# 2-dimensional grid. We will choose to represent observations in the form
# of dictionaries with keys ``"agent"`` and ``"target"``. An observation
# may look like ``{"agent": array([1, 0]), "target": array([0, 3])}``.
# Since we have 4 actions in our environment (“right”, “up”, “left”,
# “down”), we will use ``Discrete(4)`` as an action space. Here is the
# declaration of ``GridWorldEnv`` and the implementation of ``__init__``:


class GuessNumberEnv(gym.Env):
    metadata = {"render_modes": ['human'], "render_fps": 1}

    def __init__(self, algorithm, render_mode=None, min_number=1, max_number=100):
        self.algorithm = algorithm

        self._agent_number = None
        self._target_number = None

        self.min_number = min_number  # The number which is the lower threshold for the numbers range
        self.max_number = max_number  # The number which is the upper threshold for the numbers range 

        # Observations are dictionaries with the agent's and the target's numbers.
        self.observation_space = spaces.Dict(
            {
                "agent": spaces.Discrete(max_number - min_number + 1),
                "target": spaces.Discrete(max_number - min_number + 1),
            }
        )

        # The action in this env is a number from the range [min_number, max_number]
        self.action_space = spaces.Discrete(max_number - min_number + 1)

        assert render_mode is None or render_mode in self.metadata["render_modes"]
        self.render_mode = render_mode

        """
        If human-rendering is used, `self.window` will be a reference
        to the window that we draw to. `self.clock` will be a clock that is used
        to ensure that the environment is rendered at the correct framerate in
        human-mode. They will remain `None` until human-mode is used for the
        first time.
        """
        self.window = None
        self.clock = None

    # %%
    # Constructing Observations From Environment States
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    #
    # Since we will need to compute observations both in ``reset`` and
    # ``step``, it is often convenient to have a (private) method ``_get_obs``
    # that translates the environment’s state into an observation. However,
    # this is not mandatory and you may as well compute observations in
    # ``reset`` and ``step`` separately:

    def _get_obs(self):
        return {"agent": self._agent_number, "target": self._target_number}

    # %%
    # We can also implement a similar method for the auxiliary information
    # that is returned by ``step`` and ``reset``. In our case, we would like
    # to provide the manhattan distance between the agent and the target:

    def _get_info(self):
        diff = self._target_number - self._agent_number
        match self.algorithm:
            case "QLearning":
                """
                When `self._target_number` is greater than chosen return 1
                When `self._target_number` is less than chosen return -1
                When `self._target_number` is equal to chosen return 0
                """
                return {'distance': np.sign(diff)}
            case "SARSA":
                """
                When `self._target_number` is less than chosen return -2
                When `self._target_number` is greater than chosen return -1
                When `self._target_number` is equal to chosen return 0
                """
                if diff < 0:
                    return {'distance': -2, 'message': f'Chosen number ({self._agent_number}) is too big'}
                elif diff > 0:
                    return {'distance': -1, 'message': f'Chosen number ({self._agent_number}) is too small'}
                else:
                    return {'distance': 0, 'message': f'Correctly guessed number ({self._agent_number})!'}

    # %%
    # Oftentimes, info will also contain some data that is only available
    # inside the ``step`` method (e.g. individual reward terms). In that case,
    # we would have to update the dictionary that is returned by ``_get_info``
    # in ``step``.

    # %%
    # Reset
    # ~~~~~
    #
    # The ``reset`` method will be called to initiate a new episode. You may
    # assume that the ``step`` method will not be called before ``reset`` has
    # been called. Moreover, ``reset`` should be called whenever a done signal
    # has been issued. Users may pass the ``seed`` keyword to ``reset`` to
    # initialize any random number generator that is used by the environment
    # to a deterministic state. It is recommended to use the random number
    # generator ``self.np_random`` that is provided by the environment’s base
    # class, ``gymnasium.Env``. If you only use this RNG, you do not need to
    # worry much about seeding, *but you need to remember to call
    # ``super().reset(seed=seed)``* to make sure that ``gymnasium.Env``
    # correctly seeds the RNG. Once this is done, we can randomly set the
    # state of our environment. In our case, we randomly choose the agent’s
    # location and the random sample target positions, until it does not
    # coincide with the agent’s position.
    #
    # The ``reset`` method should return a tuple of the initial observation
    # and some auxiliary information. We can use the methods ``_get_obs`` and
    # ``_get_info`` that we implemented earlier for that:

    def reset(self, seed=None, options=None):
        # We need the following line to seed self.np_random
        super().reset(seed=seed)

        # Choose the agent's number uniformly at random
        self._agent_number = self.np_random.integers(self.min_number, self.max_number, dtype=int)

        # We will sample the target's number randomly until it does not coincide with the agent's number
        self._target_number = self._agent_number
        while self._target_number == self._agent_number:
            self._target_number = self.np_random.integers(self.min_number, self.max_number, dtype=int)

        observation = self._get_obs()
        info = self._get_info()

        return observation

    # %%
    # Step
    # ~~~~
    #
    # The ``step`` method usually contains most of the logic of your
    # environment. It accepts an ``action``, computes the state of the
    # environment after applying that action and returns the 5-tuple
    # ``(observation, reward, terminated, truncated, info)``. See
    # :meth:`gymnasium.Env.step`. Once the new state of the environment has
    # been computed, we can check whether it is a terminal state and we set
    # ``done`` accordingly. Since we are using sparse binary rewards in
    # ``GridWorldEnv``, computing ``reward`` is trivial once we know
    # ``done``.To gather ``observation`` and ``info``, we can again make
    # use of ``_get_obs`` and ``_get_info``:

    def step(self, action):
        # An episode is done if the agent has reached the target
        self._agent_number = action
        observation = self._get_obs()
        info = self._get_info()

        reward = info["distance"]
        terminated = not info["distance"]

        return observation, reward, terminated, False, info

    # %%
    # Rendering
    # ~~~~~~~~~

    def render(self):
        # if self.render_mode == "rgb_array":
        #     return self._render_frame()

        match self.render_mode:
            case "human":
                print(f"Agent number: {self._agent_number}, Target number: {self._target_number}, Reward: {self._get_info()['distance']}")
                return


SyntaxError: invalid syntax (3964068844.py, line 86)

In [ ]:
register(
    id="envs/GuessNumberEnv-v0",
    entry_point="envs:GuessNumberEnv",
    max_episode_steps=300,
)

## Opisy algorytmów użytych do uczenia ze wzmocnieniem

W naszych eksperymentach wykorzystaliśmy dwa algorytmy QLearning i SARSA.

### Opis działania QLearning

Część teoretyczna działania algorytmu QLearning została przez nas dokładnie opisana w ramach poprzedniego laboratorium, zatem teraz przedstawimy tylko opis działania w naszej implementacji. Przy każdym zgadywaniu miał $\epsilon$ szans na wybranie losowej liczby i $1 - \epsilon$ na wybranie wartości z QTable. To podejście niestety nie jest dobre z dwóch powodów. Pierwszym powodem jest fakt, że QTable powinno być różne dla każdej możliwej liczby. Dla poprawnego działania algorytmu konieczne byłoby znanie liczby przez agenta, co w oczywisty sposób nie ma sensu. Drugi powód związany jest z innym podejściem do uczenia. Możnaby spróbować za pomoczą wartości w QTable ograniczać przedziały, w których może znaleźć się liczba. Jest to jednak przerost formy nad treścią, ponieważ dla każdej wylosowanej liczby przeprowadzony musiałby być osobny proces uczenia, który de facto sprowadziłby się do implementacji bin search, jednak dużo mniej efektywnej i niepotrzebnie skomplikowanej. Ostatecznie udawało się zakończyć działanie algorytmu po losowym wyborze liczby.

### Opis działania SARSA

Algorytm SARSA uczy się według wzoru:

$$Q(s_t, a_t) \leftarrow (1 - \alpha)Q(s_t, a_t) + \alpha \cdot [r_{t+1} + \gamma \cdot Q(s_{t+1}, a_{t+1})],$$

gdzie:
* $Q(s_t,a_t)$: aktualna ocena funkcji wartości akcji (Q-funkcji) dla stanu $s_t$ i akcji 
$a_t$. Oznacza oczekiwaną sumę nagród uzyskanych poprzez wykonanie akcji $a_t$ w stanie $s_t$.
* $\alpha$: współczynnik uczenia.
* $r_{t+1}$: natychmiastowa nagroda.
* $\gamma$: współczynnik zmniejszający wagę przyszłych nagród.
* $Q(s_{t+1}, a_{t+1})$: ocena wartości dla przyszłego stanu po przyszłej akcji wybranej według aktualnej polityki.

Algorytm SARSA jest wariacją na temat algorytmu QLearning. Główną różnicą jest podejście *off policy* w QLearning i *on policy* w SARSA. W pierwszej polityce agent wyciąga ocenę akcji z polityki, która nie jest aktualnie w użyciu. W drugim podejściu przeciwnie, agent ocenia wartość akcji na podstawie aktualnej polityki.

Źródło: [Artykuł na GeeksForGeeks](https://www.geeksforgeeks.org/sarsa-reinforcement-learning/).

W naszym przypadku uczenie algorytmem SARSA również okazało się nieskuteczne. Algorytm ostatecznie zgadywał liczbę ale działał losowo

### Eksperymenty QLearning

In [ ]:
class QLearningAgent:
    def __init__(
            self,
            observation_space,
            action_space,
            learning_rate=0.1,
            discount_factor=0.99,
            exploration_rate=1.0,
            min_exploration_rate=0.01,
            exploration_decay_rate=0.99
    ):
        self.observation_space = observation_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.exploration_rate = exploration_rate
        self.min_exploration_rate = min_exploration_rate
        self.exploration_decay_rate = exploration_decay_rate
        self.q_table = np.zeros((action_space.n,))

    def choose_action(self):
        if uniform(0, 1) < self.exploration_rate:
            return self.action_space.sample()  # Explore action space
        else:
            return np.argmax(self.q_table)  # Exploit learned values

    def update_q_table(self, action, reward):
        old_q_value = self.q_table[action]
        next_max = np.max(self.q_table)
        new_q_value = (1 - self.learning_rate) * old_q_value + self.learning_rate * (reward + self.discount_factor * next_max)
        self.q_table[action] = new_q_value

    def decay_exploration_rate(self):
        self.exploration_rate = max(self.min_exploration_rate, self.exploration_rate * self.exploration_decay_rate)


In [5]:
env = gym.make("envs/GuessNumberEnv-v0", algorithm="QLearning", render_mode="human")
agent = QLearningAgent(env.observation_space['agent'], env.action_space)

# Train the agent
num_episodes = 1000
for episode in tqdm(range(num_episodes)):
    observation = env.reset()
    done = False
    while not done:
        action = agent.choose_action()
        next_observation, reward, done, _, _ = env.step(action)
        agent.update_q_table(action, reward)
        observation = next_observation
    agent.decay_exploration_rate()

# Test the trained agent
observation = env.reset()
done = False
while not done:
    action = agent.choose_action()
    next_observation, reward, done, _, _ = env.step(action)
    observation = next_observation
    env.render()

ModuleNotFoundError: No module named 'lab4'

### Eksperymenty SARSA

In [ ]:

class SARSAAgent:
    def __init__(self, observation_space, action_space, learning_rate=0.1, discount_factor=0.99, epsilon=0.1):
        self.observation_space = observation_space
        self.action_space = action_space
        self.learning_rate = learning_rate
        self.discount_factor = discount_factor
        self.epsilon = epsilon
        self.q_table = np.zeros((observation_space["agent"].n, action_space.n))

    def choose_action(self, observation):
        if np.random.rand() < self.epsilon:
            return np.random.randint(self.action_space.n)
        else:
            return np.argmax(self.q_table[observation["agent"]])

    def update_q_table(self, observation, action, reward, next_observation, next_action):
        current_q_value = self.q_table[observation["agent"], action]
        next_q_value = self.q_table[next_observation["agent"], next_action]
        td_target = reward + self.discount_factor * next_q_value
        td_error = td_target - current_q_value

        if reward == -1:
            # chosen number is too small, so update every possible [state, action] smaller than chosen number
            # so that we can avoid choosing smaller numbers than currently chosen one
            self.q_table[: observation["agent"] + 1, : action + 1] += self.learning_rate * td_error
            self.q_table[:, : action + 1] += self.learning_rate * td_error
            self.q_table[: observation["agent"] + 1, :] += self.learning_rate * td_error
            return
        if reward == -2:
            # chosen number is too big
            self.q_table[observation["agent"]:, action:] += self.learning_rate * td_error
            self.q_table[:, action:] += self.learning_rate * td_error
            self.q_table[observation["agent"]:, :] += self.learning_rate * td_error

            return

        self.q_table[observation["agent"], action] += self.learning_rate * td_error

    def train(self, env, episodes):
        for i in tqdm(range(episodes)):
            observation = env.reset()
            action = observation['agent']
            done = False
            while not done:
                next_observation, reward, done, _, info = env.step(action)
                # if i in [0, 50, 99]:
                #     print(f'{i} {info["message"]}')
                next_action = self.choose_action(next_observation)
                self.update_q_table(observation, action, reward, next_observation, next_action)
                observation = next_observation
                action = next_action

In [6]:
env_sarsa = gym.make("envs/GuessNumberEnv-v0", algorithm="SARSA", render_mode="human")
agent = SARSAAgent(env_sarsa.observation_space, env_sarsa.action_space)

# Train the agent
agent.train(env_sarsa, episodes=1000)

# Test the agent
observation = env_sarsa.reset()
done = False
while not done:
    action = agent.choose_action(observation)
    observation, reward, done, _, _ = env_sarsa.step(action)
    env_sarsa.render()


ModuleNotFoundError: No module named 'lab4'

## Podsumowanie

Gra Zgadnij Liczbę nie jest grą dobrą do stosowania uczenia ze wzmocnieniem. Nauka algorytmów sprowadza się do implementacji algorytmu bin search, który jest niepotrzebnie skomplikowany. Kolejnym problemem jest różnorodność środowiska. Agent w idealnej sytuacji powinien się nauczyć akcji do zgadywania konkretnej liczby, jednak wymaga to znania liczby a priori, czyli sprawia, że uczenie nie ma sensu. Naszym innym pomysłem byłoby uzależnienie nagrody od tego jak daleko jest liczba, którą zgadł agent od oczekiwanej. To podejście również jest problematyczne, ponieważ zdradza zbyt dużo informacji agentowi na starcie i sprowadza się do poznania liczby na samym początku. Uczenie ze wzmocnieniem działa dobrze w grach takich jak blackjack, gdzie możemy stworzyć odpowiednią politykę. W grze Zgadnij Liczbę nie da się określić polityki, zaś agent musiałby się nauczyć algorytmu bin search.